<a href="https://colab.research.google.com/github/realBagher/sentiment_analysis_food_classifcation/blob/main/bert_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [ ]:
import numpy as np
import pandas as pd
import re
import os
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, KFold

import keras
import tensorflow as tf
import tensorflow_datasets as tfds
# import tensorflow_text
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, GlobalAveragePooling1D, LSTM, Bidirectional
from tensorflow.python.client import device_lib
# from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import plotly.graph_objs as go
from plotly.subplots import make_subplots
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from keras.layers import Bidirectional
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,classification_report
import spacy,string
import tqdm
tqdm.tqdm.pandas()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/project2/train_data.csv")
df= df.sample(frac=1)[['reviewText','summary','overall']]
df.shape

(838944, 3)

In [ ]:
# df_5=df[df['overall']==5]
# other_df =df[df['overall']!=5]
# df=pd.concat([df_5[:20000],other_df],axis=0)
# df

In [ ]:
# from transformers import *  # this is HuggingFace library
# import tensorflow as tf
# from transformers import BertTokenizer, TFBertModel, BertConfig

In [ ]:
# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# config = BertConfig.from_pretrained("bert-base-uncased",output_hidden_states=False)
# bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)

In [ ]:
column ='overall'
colors = ['#581845','#C70039','#2E4053','#1ABC9C','#7F8C8D']
fig = make_subplots(rows=1,cols=2,
                    subplot_titles=('Countplot','Percentages'),
                    specs=[[{"type": "xy"}, {'type':'domain'}]])

fig.add_trace(go.Bar( y = df[column].value_counts().values.tolist(),
                  x = [str(i) for i in df[column].value_counts().index],
                  text = df[column].value_counts().values.tolist(),
                  textfont = dict(size=15),
                  name = column,
                  textposition = 'auto',
                  showlegend=False,
                  marker=dict(color = colors,
                              line=dict(color='#DBE6EC',
                                        width=1))),
          row = 1, col = 1)

fig.add_trace(go.Pie(labels= df[column].value_counts().keys(),
                     values= df[column].value_counts().values,
                     textfont = dict(size = 20),
                     textposition='auto',
                     showlegend = False,
                     name = column,
                     marker=dict(colors=colors)),
              row = 1, col = 2)


fig.update_layout(title={'text': column,
                         'y':0.5,
                         'x':.5,
                         'xanchor': 'center',
                         'yanchor': 'bottom'},
                  template='plotly_white')

iplot(fig)

In [ ]:


import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
df['summary']=df['summary'].astype(str)

In [ ]:
df['review_clean'] = df['reviewText'].progress_apply(clean_text)
df['summary_clean'] = df['summary'].progress_apply(clean_text)

100%|██████████| 838944/838944 [00:03<00:00, 235081.36it/s]


In [ ]:
# df.to_csv('/kaggle/working/clean_train.csv',encoding='utf-8')

In [ ]:
df['total_text']=df['summary']+'.'+df['review_clean']
# df['total_text']=df['review_clean']

# sentences = df['total_text']
c

699277    I received these today and they sound fantasti...
88767     The speakers are Great and reproduce quality s...
501031    Equipment formats the card into fat16, complet...
591015    I love the color.bought ipod son sending one b...
82300     Love it mainly for the inexpensive price.love ...
                                ...                        
466539    Love it! Easy to install, sturdy, well-designe...
427732    Worth every penny.ordered one last ipad well m...
784993    Good Throughput, But Drop The UTM Services.one...
428609    Improved over gen 1, great sound..heos 3 gen 1...
734219    did not work for my usb mouse to PS2 mouse por...
Name: total_text, Length: 838944, dtype: object

In [ ]:
target = np.array(pd.get_dummies(df['overall']))

**Bert**

In [ ]:
import pandas as pd
import gzip
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import torch
import numpy as np

In [ ]:
new_df= df

In [ ]:
new_df.dropna(subset=['reviewText', 'overall'], inplace=True)
new_df['overall'] = new_df['overall'] - 1

In [ ]:
new_df.overall.value_counts()

overall
4    461485
3    156514
0     82950
2     81239
1     56756
Name: count, dtype: int64

In [ ]:
df_test = new_df[['total_text','overall']].iloc[:50000, :].reset_index(drop=True)
df_test.rename({'total_text':'text','overall':'label'},axis=1,inplace=True)
df_train = new_df[['total_text','overall']].iloc[50000:, :].reset_index(drop=True)
df_train.rename({'total_text':'text','overall':'label'},axis=1,inplace=True)

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 788944
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(batch):
#     tokenized_inputs =tokenizer(batch['text'],  truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    tokenized_inputs =tokenizer(batch['text'],  truncation=True,padding='max_length', max_length=100)

    tokenized_inputs['labels']=torch.tensor(batch['label'])
    return tokenized_inputs


# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/788944 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 788944
})

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
# df_train.column_names


In [ ]:
# tokenized_train = tokenized_train.remove_columns(['text'])
# tokenized_test = tokenized_test.remove_columns(['text'])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
#     num_labels=len(np.unique(df['overall']))
    num_labels=5
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10,
#     fp16=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
#     precision = precision_score(labels, preds, average='weighted')
#     recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
        'f1': f1
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7219622135162354,
 'eval_accuracy': 0.71318,
 'eval_f1': 0.6940774495936938,
 'eval_runtime': 179.7448,
 'eval_samples_per_second': 278.172,
 'eval_steps_per_second': 4.351,
 'epoch': 1.0}

In [ ]:
trainer.save_model('/kaggle/working/final_Model')

In [ ]:
!zip -r /kaggle/working/final_Model_1epoch.zip 'final_Model'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: final_Model/ (stored 0%)
  adding: final_Model/model.safetensors (deflated 7%)
  adding: final_Model/training_args.bin (deflated 51%)
  adding: final_Model/kaggle/ (stored 0%)
  adding: final_Model/kaggle/working/ (stored 0%)
  adding: final_Model/kaggle/working/final_Model/ (stored 0%)
  adding: final_Model/kaggle/working/final_Model/model.safetensors (deflated 7%)
  adding: final_Model/kaggle/working/final_Model/training_args.bin (deflated 51%)
  adding: final_Model/kaggle/working/final_Model/config.json (deflated 53%)
  adding: final_Model/config.json (deflated 53%)


In [ ]:
!unzip -o /kaggle/working/final_Model_1epoch.zip -d  /kaggle/working/final_Model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


unzip:  cannot find or open /kaggle/working/final_Model_1epoch.zip, /kaggle/working/final_Model_1epoch.zip.zip or /kaggle/working/final_Model_1epoch.zip.ZIP.


In [ ]:
# !pip install evaluate

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate

In [ ]:
classifier = pipeline("text-classification", model='/kaggle/input/models/kaggle/working/final_Model', tokenizer=tokenizer,truncation=True,device=0,
                      batch_size=4)

In [ ]:
classifier("best")

[{'label': 'LABEL_4', 'score': 0.5525587201118469}]

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate
# data = load_dataset("imdb", split="test").shuffle().select(range(1000))
metric = evaluate.load("accuracy")
task_evaluator = evaluator("text-classification")
results = task_evaluator.compute(model_or_pipeline=classifier, data=test_dataset, metric=metric,
                       label_mapping={"LABEL_0": 0, "LABEL_1": 1,'LABEL_2':2,'LABEL_3':3,'LABEL_4':4},)
results

In [ ]:
trainer.save_model('spam_model')